In [ ]:
import hyperspy.api as hys
import numpy as np
import tkinter.filedialog as tkf

In [ ]:
# 1) prepare EELS data (low loss and core loss)
hl = hys.load(tkf.askopenfilename(), signal_type = "EELS")
ll = hys.load(tkf.askopenfilename(), signal_type = "EELS")
print(hl)
print(ll)

In [ ]:
hlc = hl.deepcopy()
llc = ll.deepcopy()

In [ ]:
# 2) ZLP align
llc.align_zero_loss_peak(subpixel=True, also_align=[hlc])

In [ ]:
%matplotlib qt
hlc.plot()

In [ ]:
# 4_1) Fourier ratio deconvolution
hl_deconv_fr = hlc.fourier_ratio_deconvolution(llc)

In [ ]:
ll_deconv_fr.plot()

In [ ]:
# 4_2) Fourier log deconvolution
ll_deconv_fl = llc.fourier_log_deconvolution(llc, add_zlp=True)

In [ ]:
ll_deconv_fl.plot()

In [ ]:
# 4_3) Richardson-Lucy deconvolution
ll_deconv_rl = llc.richardson_lucy_deconvolution(llc)

In [ ]:
ll_deconv_rl.plot()

In [ ]:
ll_deconv_rl.save(tkf.asksaveasfilename())

In [ ]:
# 5) add elements
hl_deconv.add_elements(("Co",))

In [ ]:
# 6) create a fitting model
model = hl_deconv.create_model(ll=llc, auto_background=False)
print(model.components)

In [ ]:
# 7) fit the model
model.multifit(kind="smart")

In [ ]:
model.plot()
model.enable_adjust_position()

In [ ]:
print(model.components.Co_L3.onset_energy.value)
print(model.components.Co_L2.onset_energy.value)

In [ ]:
model.enable_fine_structure()
model.multifit(kind="smart")

In [ ]:
model.plot()

In [ ]:
model.components.Co_L3.intensity.bmin = 0.0
model.multifit(fitter="leastsq", kind="smart", bounded=True)

In [ ]:
model.plot()

In [ ]:
roi = hys.roi.SpanROI(left=750, right=800)
model.plot()
roi.add_widget(model, axes=["Energy loss"])

In [ ]:
co_l3 = model.as_signal().isig[roi].integrate1D(axis="Energy loss")
print(co_l3.data)

In [ ]:
roi = hys.roi.SpanROI(left=750, right=800)
model.plot()
roi.add_widget(model, axes=["Energy loss"])

In [ ]:
co_l2 = model.as_signal().isig[roi].integrate1D(axis="Energy loss")
print(co_l2.data)

In [ ]:
wl_ratio = co_l3 / co_l2
print(wl_ratio.data)